In [40]:
import zipfile
#import urllib.request
import requests as req

import os
 
url = 'https://files.grouplens.org/datasets/movielens/ml-20m.zip'
filename = '../data/movielens.zip'
if not os.path.exists(filename):
    file = req.get(url, allow_redirects=True)
    open(filename, "wb").write(file.content)

    #, allow_redirects=True
    #urllib.request.urlretrieve(url, filename)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('../data')


In [41]:
#data analysis libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#ignore warnings
import warnings

warnings.filterwarnings('ignore')

In [42]:
# https://aigamer28100.github.io/Movie-Lens-Dataset-Visualization-and-Prediction/

In [43]:
mldir="../data/ml-20m/"

#import train and test CSV files
movie = pd.read_csv(mldir+"movies.csv")
ratings = pd.read_csv(mldir+"ratings.csv")
links = pd.read_csv(mldir+"links.csv")
tags = pd.read_csv(mldir+"tags.csv")
#take a look at the training data
print(movie.shape)
print(ratings.shape)
print(links.shape)
print(tags.shape,end="\n\n")
#m=pd.read_csv("movies.csv")


(27278, 3)
(20000263, 4)
(27278, 3)
(465564, 4)



In [44]:

#get a list of the features within the dataset
print("Movie : ", movie.columns,end="\n\n")
print("Rating : ", ratings.columns,end="\n\n")
print("Links : ", links.columns,end="\n\n")
print("Tags : ", tags.columns,end="\n\n")

movie.info()
ratings.info()
tags.info()

Movie :  Index(['movieId', 'title', 'genres'], dtype='object')

Rating :  Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

Links :  Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

Tags :  Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entrie

In [45]:
movie.head(10)



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [46]:
movie.loc[movie['genres'].str.contains("(no genres listed)")]

,movieId,title,genres
16574,83773,Away with Words (San tiao ren) (1999),(no genres listed)
16589,83829,Scorpio Rising (1964),(no genres listed)
16764,84768,Glitterbug (1994),(no genres listed)
17080,86493,"Age of the Earth, The (A Idade da Terra) (1980)",(no genres listed)
17243,87061,Trails (Veredas) (1978),(no genres listed)
...,...,...,...
27216,131082,Playground (2009),(no genres listed)
27229,131108,The Fearless Four (1997),(no genres listed)
27258,131166,WWII IN HD (2009),(no genres listed)
27261,131172,Closed Curtain (2013),(no genres listed)


In [47]:
links.head(3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [48]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819


In [49]:
tags.head(3)


,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079


In [50]:
del tags, links

In [51]:
# Droping the timestamp column from ratings and tags file
ratings.drop(columns='timestamp',inplace=True)
#tags.drop(columns='timestamp',inplace=True)

In [52]:
all_categories = set(cat for sublist in movie['genres'].str.split("|") for cat in sublist)
print(all_categories)

{'Comedy', 'Thriller', 'Romance', 'Children', 'Sci-Fi', 'Action', 'Mystery', '(no genres listed)', 'War', 'Musical', 'Drama', 'Film-Noir', 'IMAX', 'Western', 'Horror', 'Animation', 'Fantasy', 'Adventure', 'Crime', 'Documentary'}


In [53]:
for category in all_categories:  
    movie[category] = movie['genres'].apply(lambda x: 1 if category in x else 0)

# Drop the original 'Categories' column
movie = movie.drop('genres', axis=1)
    

In [54]:
x=ratings.groupby('movieId').rating.mean()


In [55]:
type(x)

pandas.core.series.Series

In [56]:
movie.columns

Index(['movieId', 'title', 'Comedy', 'Thriller', 'Romance', 'Children',
       'Sci-Fi', 'Action', 'Mystery', '(no genres listed)', 'War', 'Musical',
       'Drama', 'Film-Noir', 'IMAX', 'Western', 'Horror', 'Animation',
       'Fantasy', 'Adventure', 'Crime', 'Documentary'],
      dtype='object')

In [57]:
type(movie['IMAX'])

pandas.core.series.Series

In [58]:
movie['IMAX']

0        0
1        0
2        0
3        0
4        0
        ..
27273    0
27274    0
27275    0
27276    0
27277    0
Name: IMAX, Length: 27278, dtype: int64

In [59]:
x

movieId
1         3.921240
2         3.211977
3         3.151040
4         2.861393
5         3.064592
            ...   
131254    4.000000
131256    4.000000
131258    2.500000
131260    3.000000
131262    4.000000
Name: rating, Length: 26744, dtype: float64

In [60]:
#Add a Column `rating` in movie DF and assign them with the Mean Movie Rating for that Movie.
x=ratings.groupby('movieId').rating.mean()
movie = pd.merge(movie,x,how='outer',on='movieId')

In [61]:
movie['rating'].fillna('0',inplace=True)


In [62]:
movie.head()

,movieId,title,Comedy,Thriller,Romance,Children,Sci-Fi,Action,Mystery,(no genres listed),...,Film-Noir,IMAX,Western,Horror,Animation,Fantasy,Adventure,Crime,Documentary,rating
0,1,Toy Story (1995),1,0,0,1,0,0,0,0,...,0,0,0,0,1,1,1,0,0,3.92124
1,2,Jumanji (1995),0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,3.211977
2,3,Grumpier Old Men (1995),1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.15104
3,4,Waiting to Exhale (1995),1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.861393
4,5,Father of the Bride Part II (1995),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.064592


In [63]:
#importing necessary packages for model prediction and evaluation
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split

In [64]:
movie.columns

Index(['movieId', 'title', 'Comedy', 'Thriller', 'Romance', 'Children',
       'Sci-Fi', 'Action', 'Mystery', '(no genres listed)', 'War', 'Musical',
       'Drama', 'Film-Noir', 'IMAX', 'Western', 'Horror', 'Animation',
       'Fantasy', 'Adventure', 'Crime', 'Documentary', 'rating'],
      dtype='object')

In [65]:
#split the data into features and results
X = movie[movie.columns[2:-1]]
y = movie[movie.columns[-1]]

In [66]:
X.columns

Index(['Comedy', 'Thriller', 'Romance', 'Children', 'Sci-Fi', 'Action',
       'Mystery', '(no genres listed)', 'War', 'Musical', 'Drama', 'Film-Noir',
       'IMAX', 'Western', 'Horror', 'Animation', 'Fantasy', 'Adventure',
       'Crime', 'Documentary'],
      dtype='object')

In [67]:
X_1 , X_test , y_1 , y_test = train_test_split(X ,y ,test_size=0.20 ,random_state = 1 ,shuffle = True)

X_train , X_validation , y_train , y_validation = train_test_split(X_1 ,y_1 ,test_size=0.20 ,random_state = 1 ,shuffle = True)


In [68]:
X_train.shape

(17457, 20)

In [69]:
X_train.head()

,Comedy,Thriller,Romance,Children,Sci-Fi,Action,Mystery,(no genres listed),War,Musical,Drama,Film-Noir,IMAX,Western,Horror,Animation,Fantasy,Adventure,Crime,Documentary
359,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17425,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0
5307,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5990,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
7787,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [70]:
model = DecisionTreeRegressor(max_depth=10,random_state=42)
model.fit(X_1,y_1)
print(mean_squared_error(model.predict(X_test),y_test))

0.5468541227981384


In [71]:
print(mean_squared_error(model.predict(X_test),y_validation))


0.6420469572250624


In [107]:
tree_params = {'max_depth':[3,5,10]}
DTCG = GridSearchCV(DecisionTreeRegressor(random_state=42), tree_params, cv=5,scoring='neg_mean_squared_error'  )
DTCG.fit(X_1,y_1)
DTC=DTCG.best_estimator_
score=DTCG.best_score_
print(score)

-0.5840912333176995


In [90]:

mean_squared_error(DTC.predict(X_validation),y_validation)


0.6285092705324791

In [74]:
mean_squared_error(DTC.predict(X_test),y_test)

0.5717914257637114

In [75]:
mean_squared_error(DTCG.predict(X_test),y_test)


0.5717914257637114

In [76]:
x.shape

(26744,)

In [77]:
DTCG.best_params_

{'max_depth': 5}

In [78]:
X.shape

(27278, 20)

In [91]:
y.head()

0     3.92124
1    3.211977
2     3.15104
3    2.861393
4    3.064592
Name: rating, dtype: object

In [92]:
type(y_test)

pandas.core.series.Series

In [93]:
y_pred_1=y_test.copy()

In [103]:
np.mean(y_test.values)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [100]:
np.mean(y_test.value)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [104]:
y_test.mean()

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [ ]:
mean_squared_error(y_test, 